In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0532.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0282.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_1401.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0914.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0691.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0972.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0818.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0463.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0930.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_1323.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_1347.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_1354.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0704.jpg
/kaggle/input/brain-tumor-mri-dataset/Training/pituitary/Tr-pi_0

In [13]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow_hub as hub

In [7]:
train_path = "/kaggle/input/brain-tumor-mri-dataset/Training"
test_path = "/kaggle/input/brain-tumor-mri-dataset/Testing"

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(train_path,
                                                     image_size=(255, 255),
                                                     batch_size=32)

test_ds = tf.keras.utils.image_dataset_from_directory(test_path,
                                                     image_size=(255, 255),
                                                     batch_size=32)

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [9]:
class_names = train_ds.class_names

In [10]:
class_names

['glioma', 'meningioma', 'notumor', 'pituitary']

In [11]:
resize_and_rescale = tf.keras.Sequential([
    layers.Rescaling(1/255),
    layers.Resizing(255, 255)
])

In [15]:
model = tf.keras.Sequential([
    resize_and_rescale,
    hub.KerasLayer("https://www.kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/101-classification/versions/2", trainable=False),
    layers.Dense(len(class_names), activation="softmax")
])
model.build([None, 224, 224, 3])  # Batch input shape.


In [16]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=tf.keras.optimizers.Adam(),
             metrics=["accuracy"])

In [18]:
model_history = model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
179/179 [==============================] - 45s 248ms/step - loss: 0.2188 - accuracy: 0.9181 - val_loss: 0.2881 - val_accuracy: 0.8833
Epoch 2/5
179/179 [==============================] - 41s 228ms/step - loss: 0.2098 - accuracy: 0.9247 - val_loss: 0.2825 - val_accuracy: 0.8871
Epoch 3/5
179/179 [==============================] - 42s 231ms/step - loss: 0.1979 - accuracy: 0.9245 - val_loss: 0.2676 - val_accuracy: 0.8871
Epoch 4/5
179/179 [==============================] - 41s 227ms/step - loss: 0.1912 - accuracy: 0.9303 - val_loss: 0.2632 - val_accuracy: 0.8871
Epoch 5/5
179/179 [==============================] - 41s 227ms/step - loss: 0.1849 - accuracy: 0.9282 - val_loss: 0.2590 - val_accuracy: 0.8947


In [19]:
model.save("model.h5")